### **1. Imporitng Libraries**

In [2]:
import pandas as pd
import pyodbc
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [3]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\habib\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

### **2. Fetching Data From the SQL Server**

In [4]:
def fetch_data_from_sql():
    conn_str = (
        "Driver={SQL Server};"
        "Server=HABIBASDEVICE\SQLEXPRESS;"
        "Database=PortfolioProject_MarketingAnalytics;"
        "Trusted_Connection=yes;"
    )
    conn = pyodbc.connect(conn_str)
    query = "SELECT ReviewID, CustomerID, ProductID, ReviewDate, Rating, ReviewText FROM customer_reviews"
    df = pd.read_sql(query, conn)
    conn.close()
    return df

<>:4: SyntaxWarning: invalid escape sequence '\S'
<>:4: SyntaxWarning: invalid escape sequence '\S'
C:\Users\habib\AppData\Local\Temp\ipykernel_22752\121646942.py:4: SyntaxWarning: invalid escape sequence '\S'
  "Server=HABIBASDEVICE\SQLEXPRESS;"


In [5]:
customer_reviews_df = fetch_data_from_sql()

C:\Users\habib\AppData\Local\Temp\ipykernel_22752\121646942.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


### **3. Defining Functions for Sentiment Analysis**

*Initializing VADER sentiment intensity analyzer*

In [6]:
sia = SentimentIntensityAnalyzer()

In [7]:
def calculate_sentiment(review):
    sentiment = sia.polarity_scores(review)
    return sentiment['compound']    

*categorizing sentiments using **both sentiment scores and review ratings***

In [8]:
def categorize_sentiment(score, rating):
    if score > 0.05: # +ve sentiment
        if rating >= 4:
            return 'Positive'
        elif rating == 3:
            return 'Mixed Positive'
        else:
            return 'Mixed Negative'
    elif score < -0.05: # -ve sentiment
        if rating <= 2:
            return 'Negative'
        elif rating == 3:
            return 'Mixed Negative'
        else:
            return 'Mixed Positive'
    else:
        if rating >= 4:
            return 'Positive'
        elif rating <=2:
            return 'Negative'
        else:
            return 'Neutral'

*bucketing sentiment scores into text ranges*

In [9]:
def sentiment_bucket(score):
    if score >= 0.5:
        return '0.5 to 1.0' # strongly positive
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49' # weakly positive
    elif -0.5 <= score < 0.0:
        return '-0.49 to 0.0' # weakly negative
    else:
        return '-1.0 to -0.5' # strongly negative

*applying sentiment categorization using **both text and rating***

In [10]:
customer_reviews_df['SentimentScore'] = customer_reviews_df['ReviewText'].apply(calculate_sentiment)

In [11]:
customer_reviews_df['SentimentCategory'] = customer_reviews_df.apply(
    lambda row: categorize_sentiment(row['SentimentScore'], row['Rating']),
    axis=1
)

In [12]:
customer_reviews_df['SentimentBucket'] = customer_reviews_df['SentimentScore'].apply(sentiment_bucket)

### **4. Reviewing Data**

In [13]:
customer_reviews_df.head()

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText,SentimentScore,SentimentCategory,SentimentBucket
0,1,77,18,2023-12-23,3,"Average experience, nothing special.",-0.3089,Mixed Negative,-0.49 to 0.0
1,2,80,19,2024-12-25,5,The quality is top-notch.,0.0000,Positive,0.0 to 0.49
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.,0.0000,Positive,0.0 to 0.49
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper.",0.2382,Mixed Positive,0.0 to 0.49
4,5,64,2,2023-07-16,3,"Average experience, nothing special.",-0.3089,Mixed Negative,-0.49 to 0.0


### **5. Exporting Data as a CSV File**

In [14]:
customer_reviews_df.to_csv(r'C:\Users\habib\Desktop\Marketing Analytics\customer_reviews_with_sentiment.csv', index=False)